## Import libraries and dependencies


In [1]:

import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da

import datetime as dt

import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [2]:
portfolio_choice = ""
while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio - type 2\n"
        )
                              )
    except Exception:
        print('Error: Invalid Choice.\n')
        
# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")


# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)

valid_tickers = list(portfolio_df.columns.levels[0])

# Kepe only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in valid_tickers]


# Print portfolio data for visual confirmation
if portfolio_choice == 1: 
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = int()
    for ticker, units in existing_portfolio.items():
        value = portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * units
        print(f"Value of {units} {ticker}: ${value:.2f}")
        total_value += value
    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${portfolio_value:.2f}\n")

print(
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")
    

To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio - type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  xdc
Ticker:  ada
Ticker:  hbar
Ticker:  xrp
Ticker:  get
Ticker:  link
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 26900


--------------------------
Hypothetical Portfolio:
['ADA', 'GET', 'HBAR', 'LINK', 'XDC', 'XRP']
Investment amount:
$26900.00

To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-09-18
--------------------------


In [3]:
portfolio_df.tail()


XDC-USD                                                       \
                open      high       low     close    volume daily_return   
Date                                                                        
2022-01-05  0.091835  0.099741  0.091486  0.094151  12384408     0.024951   
2022-01-06  0.094158  0.094417  0.090549  0.092281   8027778    -0.019862   
2022-01-07  0.092271  0.092310  0.086859  0.086859   7796199    -0.058755   
2022-01-08  0.086866  0.088330  0.082103  0.083637   8327918    -0.037095   
2022-01-09  0.083642  0.088107  0.083002  0.087459   6420355     0.045697   

             ADA-USD                                ...   GET-USD            \
                open      high       low     close  ...       low     close   
Date                                                ...                       
2022-01-05  1.311944  1.350875  1.202679  1.236002  ...  1.861175  1.898432   
2022-01-06  1.235209  1.294576  1.193416  1.279782  ...  1.750767  1.820701   
2022-01-07  1.279917  1.282408  1.193237  1.211547  ...  1.615634  1.668551   
2022-01-08  1.211468  1.252670  1.138116  1.187512  ...  1.610123  1.651399   
2022-01-09  1.187693  1.204036  1.153479  1.173860  ...  1.643780  1.716909   

                                    LINK-USD                                   \
              volume daily_return       open       high        low      close   
Date                                                                            
2022-01-05  148022.0    -0.061214  23.525679  27.287588  22.831673  25.447802   
2022-01-06  150720.0    -0.040945  25.436605  25.919743  23.545397  25.377213   
2022-01-07  229396.0    -0.083567  25.373320  27.132687  22.907932  25.960640   
2022-01-08   84846.0    -0.010280  25.962074  27.177170  23.810572  25.474800   
2022-01-09  117726.0     0.039669  25.475780  28.511189  25.295010  27.566217   

                                     
                volume daily_return  
Date                                 
2022-01-05  4041232729     0.081695  
2022-01-06  3194740641    -0.002774  
2022-01-07  3714503464     0.022990  
2022-01-08  2828318535    -0.018714  
2022-01-09  3305192981     0.082097  

[5 rows x 36 columns]

## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [4]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adjusted_sortino': adjusted_sortino,
    'gain_pain_ratio': gain_pain_ratio,
    }
)

# Calculate proportion scores for each risk-reward metric

ratios_prop_score = da.calculate_proportion_score(ratios_df)

sharpe


ADA-USD     1.7618
GET-USD     1.2179
HBAR-USD    1.0234
LINK-USD    1.6109
XDC-USD     1.7791
XRP-USD     0.9501
dtype: float64

In [5]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order
  
da.sharpe_portfolio(sharpe, ratios_prop_score, portfolio_value)

print("\nMetric: Sortino Ratio")
print("--------------")
sortino.sort_values(ascending = False)
for ticker, ratio in sortino.iteritems():
    print(f"{ticker}: {ratio:.2f}")

print("\nMetric: Adjusted Sortino Ratio")
print("--------------")
adjusted_sortino.sort_values(ascending = False)
for ticker, ratio in adjusted_sortino.iteritems():
    print(f"{ticker}: {ratio:.2f}")
    
print("\nMetric: Gain-to-Pain Ratio")
print("--------------")
for ticker, ratio in gain_pain_ratio.items():
    print(f"{ticker}: {ratio:.2f}")



Portfolio allocation recommendations
Based on historical returns from 2019-09-18 to 2022-01-09
Total portfolio value: $26900.00

Metric: Sharpe Ratio
--------------
XDC-USD: 1.78
Recommended percentage of total portfolio: 21.32%
Recommended value allocation = $5736.14

ADA-USD: 1.76
Recommended percentage of total portfolio: 21.12%
Recommended value allocation = $5680.36

LINK-USD: 1.61
Recommended percentage of total portfolio: 19.31%
Recommended value allocation = $5193.84

GET-USD: 1.22
Recommended percentage of total portfolio: 14.60%
Recommended value allocation = $3926.73

HBAR-USD: 1.02
Recommended percentage of total portfolio: 12.27%
Recommended value allocation = $3299.63

XRP-USD: 0.95
Recommended percentage of total portfolio: 11.39%
Recommended value allocation = $3063.30


Metric: Sortino Ratio
--------------
ADA-USD: 2.86
GET-USD: 1.96
HBAR-USD: 1.89
LINK-USD: 2.44
XDC-USD: 3.56
XRP-USD: 1.55

Metric: Adjusted Sortino Ratio
--------------
ADA-USD: 2.02
GET-USD: 1.39
HBAR

In [15]:
sharpe.sort_values(ascending = False)

XDC-USD     1.7653
ADA-USD     1.7403
LINK-USD    1.5938
VET-USD     1.5817
HBAR-USD    1.0104
XRP-USD     0.9355
dtype: float64